In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
%matplotlib inline
import statsmodels.api as sm
from warnings import filterwarnings
filterwarnings('ignore')

In [ ]:
import pymysql

conn = pymysql.connect(host='127.0.0.1',
                      user='root',
                      password='maria',
                      database='aps',
                      port=3306,
                      charset='utf8',
                      autocommit=True,
                      cursorclass=pymysql.cursors.DictCursor
                      )
cursor = conn.cursor()

sql = """ select * from sales_new """

cursor.execute(sql)
row = cursor.fetchall() 
conn.close()

co_df = pd.DataFrame(row)


#데이터 불러오기

In [ ]:
co_df

,orderID,soldDate,custID,prodID,prodName,weekday,siteBusName,region,d_day_2_tem,d_day_2_hum,d_day_2_wind,d_day_2_tem_rainamount,d_day_2_rainhour,order_quant,sold_quant,week_seq
0,AE21030434,2021-03-07,2007400,SSA013035307,AE,목,삼표,풍납,6.6,58.5,2.2,0.0,0.0,532,1000,10
1,HR1021030423,2021-03-07,2001500,PEMA363048300,PEMA-HR1000,목,제일,홍성,5.5,65.0,0.9,0.0,0.0,5184,6000,10
2,HR1021030426,2021-03-07,2001600,PEMA093001100,PEMA-HR1000,목,동해,동해,8.1,62.1,2.1,0.0,0.0,10032,8000,10
3,HR1021030427,2021-03-07,2001700,PEMA093001100,PEMA-HR1000,목,동해,동해,8.1,62.1,2.1,0.0,0.0,1344,2000,10
4,HR1521030432,2021-03-07,2002800,PEMA053046500,PEMA-HR1500,목,신일,송파,6.6,58.5,2.2,0.0,0.0,6260,5000,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24211,HR1021032444_210127,2021-01-30,2002400,PEMA023024801,PEMA-HR1000,수,동진,수원,5.0,89.0,2.2,1.7,5.1,3259,3650,4
24212,HR1521032423_210127,2021-01-30,2003100,PEMA013082100,PEMA-HR1500,수,대원씨앤엠,울산,10.6,75.9,2.5,12.1,8.3,11563,10950,4
24213,AE21032452_210127,2021-01-30,2006606,SSA073046500,AE,수,신일,송파,5.0,74.5,2.7,0.7,0.8,1892,1460,4
24214,HR1021032428_210127,2021-01-30,2001400,PEMA013128300,PEMA-HR1000,수,경연,원주,5.0,75.6,1.2,1.9,0.0,1682,1460,4


#컬럼명 변경

In [ ]:
co_df.rename(columns={'soldDate': 'Requested_Date'}, inplace=True)
co_df.rename(columns={'orderID': 'Order_Number' , 'sold_quant':'Requested_Quantity', 'prodName':'PRODNAME'}, inplace=True)
co_df.head()

,Order_Number,Requested_Date,custID,prodID,PRODNAME,weekday,siteBusName,region,d_day_2_tem,d_day_2_hum,d_day_2_wind,d_day_2_tem_rainamount,d_day_2_rainhour,order_quant,Requested_Quantity,week_seq
0,AE21030434,2021-03-07,2007400,SSA013035307,AE,목,삼표,풍납,6.6,58.5,2.2,0.0,0.0,532,1000,10
1,HR1021030423,2021-03-07,2001500,PEMA363048300,PEMA-HR1000,목,제일,홍성,5.5,65.0,0.9,0.0,0.0,5184,6000,10
2,HR1021030426,2021-03-07,2001600,PEMA093001100,PEMA-HR1000,목,동해,동해,8.1,62.1,2.1,0.0,0.0,10032,8000,10
3,HR1021030427,2021-03-07,2001700,PEMA093001100,PEMA-HR1000,목,동해,동해,8.1,62.1,2.1,0.0,0.0,1344,2000,10
4,HR1521030432,2021-03-07,2002800,PEMA053046500,PEMA-HR1500,목,신일,송파,6.6,58.5,2.2,0.0,0.0,6260,5000,10


In [ ]:
# 시계열분석에 사용되는 칼럼
co_df = co_df[['Order_Number', 'PRODNAME','Requested_Quantity','Requested_Date']]
co_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24216 entries, 0 to 24215
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Order_Number        24216 non-null  object
 1   PRODNAME            24216 non-null  object
 2   Requested_Quantity  24216 non-null  object
 3   Requested_Date      24216 non-null  object
dtypes: object(4)
memory usage: 756.9+ KB


In [ ]:
# object 타입을 datetime 타입으로 변환시켜줌 
co_df['Requested_Date'] = pd.to_datetime(co_df['Requested_Date'])
co_df['Requested_Quantity'] = co_df['Requested_Quantity'].astype('float')
co_df.head()

,Order_Number,PRODNAME,Requested_Quantity,Requested_Date
0,AE21030434,AE,1000.0,2021-03-07
1,HR1021030423,PEMA-HR1000,6000.0,2021-03-07
2,HR1021030426,PEMA-HR1000,8000.0,2021-03-07
3,HR1021030427,PEMA-HR1000,2000.0,2021-03-07
4,HR1521030432,PEMA-HR1500,5000.0,2021-03-07


#Nan 값 확인

In [ ]:
co_df.isnull().sum()

Order_Number          0
PRODNAME              0
Requested_Quantity    0
Requested_Date        0
dtype: int64

In [ ]:
df = co_df.copy()
df

,Order_Number,PRODNAME,Requested_Quantity,Requested_Date
0,AE21030434,AE,1000.0,2021-03-07
1,HR1021030423,PEMA-HR1000,6000.0,2021-03-07
2,HR1021030426,PEMA-HR1000,8000.0,2021-03-07
3,HR1021030427,PEMA-HR1000,2000.0,2021-03-07
4,HR1521030432,PEMA-HR1500,5000.0,2021-03-07
...,...,...,...,...
24211,HR1021032444_210127,PEMA-HR1000,3650.0,2021-01-30
24212,HR1521032423_210127,PEMA-HR1500,10950.0,2021-01-30
24213,AE21032452_210127,AE,1460.0,2021-01-30
24214,HR1021032428_210127,PEMA-HR1000,1460.0,2021-01-30


-주단위로 묶으면 데이터 맨 첫부분, 맨끝부분의 데이터가 적기때문에 처리-


In [ ]:
co_df = df[(df['Requested_Date'] < '2021-04-05')&(df['Requested_Date'] >= '2018')]

In [ ]:

demand_part1 = co_df["Requested_Quantity"].values.tolist()
neg_demand1=[]
for x in demand_part1:
    if x <=0:
        neg_demand1.append(x)
print(len(neg_demand1))

0


In [ ]:
df_prod = pd.DataFrame(co_df.PRODNAME.value_counts())
#df_prod.set_index()
df_prod.to_csv('prod.csv')
df_prod = pd.read_csv('prod.csv')
df_prod.rename(columns={'Unnamed: 0': 'PRODNAME' , 'PRODNAME':'COUNT'}, inplace=True)
df_prod.head()

,PRODNAME,COUNT
0,PEMA-HR1000,8454
1,CSA5000,3397
2,PEMA-HR1500,3113
3,PEMA-SR2000,1668
4,PEMA-PR1000,1226


# 주 단위로 자료의 수주량을 파악

#제품별 시계열 분석 및 예측 => 27개 상품을 각각 돌리기 때문에 시간이 오래걸림(30분 이상 소요됨). 결과는 prediction 테이블에 저장.

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
import itertools
from pmdarima import auto_arima
from statsmodels.tsa.stattools import adfuller
import statsmodels.api as sm

for i in range(0, len(df_prod)):
    print(df_prod['PRODNAME'][i])
    co_df_part = co_df.loc[co_df["PRODNAME"]==df_prod['PRODNAME'][i]]
    co_df_part_1 = co_df_part.copy()
    co_df_part_1 = co_df_part_1.set_index(["Requested_Date"])
    co_df_part_1_daily = co_df_part_1.resample('W').sum()
    co_df_part_1.reset_index(inplace=True)
    co_df_part_1 = co_df_part_1.groupby('Requested_Date').sum()

    #nan값처리
    demand_part2 = co_df_part_1_daily["Requested_Quantity"].values.tolist()
    neg_demand2=[]
    for x in demand_part2:
        if x <=0:
            neg_demand2.append(x)
    print(len(neg_demand2))

    #모델링
    train01 = co_df_part_1_daily['Requested_Quantity'][:-24]
    test01 = co_df_part_1_daily['Requested_Quantity'][-24:]

    auto_arima_model = auto_arima(train01, start_p = 1, start_q =1,
                                 max_p = 2 , max_q = 2, m = 12,
                                 start_P = 1,max_P =2 , start_Q = 1, max_Q = 2,
                                 seasonal = True, d = 1, D =1 , trace = True,
                                 error_action = 'ignore',
                                 suppress_warnings = True,
                                 stepwise = False)
    auto_arima_model.summary()

    if (df_prod['COUNT'][i] > 200):
        mod01 = sm.tsa.statespace.SARIMAX(train01,
                                        order=(0,1,0),
                                        seasonal_order=(1, 1, 1,12),
                                        enforce_stationarity=False,
                                        enforce_invertibility=False)
    else:
        mod01 = sm.tsa.statespace.SARIMAX(train01,
                                        order=(0,0,0),
                                        seasonal_order=(1, 1, 1,12),
                                        enforce_stationarity=False,
                                        enforce_invertibility=False)

    results01 = mod01.fit()

    #예측값
    df_pred01 = pd.DataFrame(results01.predict(start = test01.index[-1], end = '2021-10', dynamic = True))
    df_pred01 = df_pred01.reset_index()
    df_pred01.rename(columns={'index':'pred_date'}, inplace=True)
    
    conn = pymysql.connect(host='127.0.0.1',
                      user='root',
                      password='maria',
                      database='aps',
                      port=3306,
                      charset='utf8',
                      autocommit=True,
                      cursorclass=pymysql.cursors.DictCursor
                      )
    cursor = conn.cursor()

    sql_ins = """INSERT INTO prediction(preddate, prodname, pred_quant) \
                 VALUES (%s,%s,%s) """

    for j in range(0, len(df_pred01)):
        cursor.execute(sql_ins, (df_pred01['pred_date'][j].strftime('%Y-%m-%d'), df_prod['PRODNAME'][i], df_pred01['predicted_mean'][j])) 
        conn.commit()

    conn.close()


0
 ARIMA(0,1,0)(0,1,0)[12]             : AIC=2427.476, Time=0.05 sec
 ARIMA(0,1,0)(0,1,1)[12]             : AIC=2377.429, Time=0.04 sec
 ARIMA(0,1,0)(0,1,2)[12]             : AIC=2353.253, Time=0.14 sec
 ARIMA(0,1,0)(1,1,0)[12]             : AIC=inf, Time=0.20 sec
 ARIMA(0,1,0)(1,1,1)[12]             : AIC=2301.567, Time=0.17 sec
 ARIMA(0,1,0)(1,1,2)[12]             : AIC=inf, Time=0.98 sec
 ARIMA(0,1,0)(2,1,0)[12]             : AIC=2267.912, Time=0.51 sec
 ARIMA(0,1,0)(2,1,1)[12]             : AIC=inf, Time=1.02 sec
 ARIMA(0,1,0)(2,1,2)[12]             : AIC=inf, Time=0.99 sec
 ARIMA(0,1,1)(0,1,0)[12]             : AIC=2417.060, Time=0.03 sec
 ARIMA(0,1,1)(0,1,1)[12]             : AIC=2368.921, Time=0.08 sec
 ARIMA(0,1,1)(0,1,2)[12]             : AIC=2359.742, Time=0.17 sec
 ARIMA(0,1,1)(1,1,0)[12]             : AIC=2347.853, Time=0.13 sec
 ARIMA(0,1,1)(1,1,1)[12]             : AIC=2349.438, Time=0.24 sec
 ARIMA(0,1,1)(1,1,2)[12]             : AIC=2351.278, Time=0.32 sec
 ARIMA(0,1,1)

 ARIMA(2,1,0)(2,1,1)[12]             : AIC=2145.654, Time=0.46 sec
 ARIMA(2,1,1)(0,1,0)[12]             : AIC=2189.255, Time=0.12 sec
 ARIMA(2,1,1)(0,1,1)[12]             : AIC=2155.352, Time=0.21 sec
 ARIMA(2,1,1)(0,1,2)[12]             : AIC=2148.762, Time=0.69 sec
 ARIMA(2,1,1)(1,1,0)[12]             : AIC=2141.211, Time=0.38 sec
 ARIMA(2,1,1)(1,1,1)[12]             : AIC=2143.042, Time=0.55 sec
 ARIMA(2,1,1)(2,1,0)[12]             : AIC=2143.138, Time=0.75 sec
 ARIMA(2,1,2)(0,1,0)[12]             : AIC=2182.808, Time=0.09 sec
 ARIMA(2,1,2)(0,1,1)[12]             : AIC=2151.124, Time=0.46 sec
 ARIMA(2,1,2)(1,1,0)[12]             : AIC=2133.946, Time=0.96 sec

Best model:  ARIMA(0,1,0)(1,1,1)[12]          
Total fit time: 24.180 seconds
0
 ARIMA(0,1,0)(0,1,0)[12]             : AIC=2272.067, Time=0.01 sec
 ARIMA(0,1,0)(0,1,1)[12]             : AIC=2224.018, Time=0.05 sec
 ARIMA(0,1,0)(0,1,2)[12]             : AIC=2188.696, Time=0.11 sec
 ARIMA(0,1,0)(1,1,0)[12]             : AIC=inf, 

 ARIMA(1,1,2)(1,1,0)[12]             : AIC=2043.565, Time=0.39 sec
 ARIMA(1,1,2)(1,1,1)[12]             : AIC=2045.441, Time=0.55 sec
 ARIMA(1,1,2)(2,1,0)[12]             : AIC=2045.470, Time=0.61 sec
 ARIMA(2,1,0)(0,1,0)[12]             : AIC=2117.832, Time=0.03 sec
 ARIMA(2,1,0)(0,1,1)[12]             : AIC=2069.366, Time=0.11 sec
 ARIMA(2,1,0)(0,1,2)[12]             : AIC=2059.885, Time=0.22 sec
 ARIMA(2,1,0)(1,1,0)[12]             : AIC=inf, Time=0.26 sec
 ARIMA(2,1,0)(1,1,1)[12]             : AIC=2048.932, Time=0.42 sec
 ARIMA(2,1,0)(1,1,2)[12]             : AIC=2051.142, Time=0.20 sec
 ARIMA(2,1,0)(2,1,0)[12]             : AIC=inf, Time=0.46 sec
 ARIMA(2,1,0)(2,1,1)[12]             : AIC=inf, Time=0.64 sec
 ARIMA(2,1,1)(0,1,0)[12]             : AIC=2083.456, Time=0.14 sec
 ARIMA(2,1,1)(0,1,1)[12]             : AIC=2051.580, Time=0.38 sec
 ARIMA(2,1,1)(0,1,2)[12]             : AIC=2045.492, Time=0.62 sec
 ARIMA(2,1,1)(1,1,0)[12]             : AIC=inf, Time=0.89 sec
 ARIMA(2,1,1)(1

 ARIMA(1,1,1)(0,1,1)[12]             : AIC=2011.815, Time=0.17 sec
 ARIMA(1,1,1)(0,1,2)[12]             : AIC=2002.298, Time=0.25 sec
 ARIMA(1,1,1)(1,1,0)[12]             : AIC=1987.576, Time=0.14 sec
 ARIMA(1,1,1)(1,1,1)[12]             : AIC=inf, Time=0.88 sec
 ARIMA(1,1,1)(1,1,2)[12]             : AIC=inf, Time=2.50 sec
 ARIMA(1,1,1)(2,1,0)[12]             : AIC=1989.549, Time=0.26 sec
 ARIMA(1,1,1)(2,1,1)[12]             : AIC=inf, Time=1.57 sec
 ARIMA(1,1,2)(0,1,0)[12]             : AIC=2047.107, Time=0.06 sec
 ARIMA(1,1,2)(0,1,1)[12]             : AIC=2011.571, Time=0.22 sec
 ARIMA(1,1,2)(0,1,2)[12]             : AIC=inf, Time=1.19 sec
 ARIMA(1,1,2)(1,1,0)[12]             : AIC=inf, Time=0.61 sec
 ARIMA(1,1,2)(1,1,1)[12]             : AIC=inf, Time=1.60 sec
 ARIMA(1,1,2)(2,1,0)[12]             : AIC=inf, Time=1.97 sec
 ARIMA(2,1,0)(0,1,0)[12]             : AIC=2040.598, Time=0.03 sec
 ARIMA(2,1,0)(0,1,1)[12]             : AIC=1991.465, Time=0.11 sec
 ARIMA(2,1,0)(0,1,2)[12]      

 ARIMA(1,1,0)(0,1,1)[12]             : AIC=inf, Time=0.30 sec
 ARIMA(1,1,0)(0,1,2)[12]             : AIC=1863.931, Time=0.18 sec
 ARIMA(1,1,0)(1,1,0)[12]             : AIC=inf, Time=0.54 sec
 ARIMA(1,1,0)(1,1,1)[12]             : AIC=1762.424, Time=0.70 sec
 ARIMA(1,1,0)(1,1,2)[12]             : AIC=1759.941, Time=1.22 sec
 ARIMA(1,1,0)(2,1,0)[12]             : AIC=inf, Time=1.10 sec
 ARIMA(1,1,0)(2,1,1)[12]             : AIC=inf, Time=1.65 sec
 ARIMA(1,1,0)(2,1,2)[12]             : AIC=inf, Time=1.93 sec
 ARIMA(1,1,1)(0,1,0)[12]             : AIC=1908.273, Time=0.19 sec
 ARIMA(1,1,1)(0,1,1)[12]             : AIC=inf, Time=0.57 sec
 ARIMA(1,1,1)(0,1,2)[12]             : AIC=inf, Time=1.09 sec
 ARIMA(1,1,1)(1,1,0)[12]             : AIC=1761.073, Time=0.91 sec
 ARIMA(1,1,1)(1,1,1)[12]             : AIC=1751.290, Time=1.00 sec
 ARIMA(1,1,1)(1,1,2)[12]             : AIC=inf, Time=2.04 sec
 ARIMA(1,1,1)(2,1,0)[12]             : AIC=1748.668, Time=1.72 sec
 ARIMA(1,1,1)(2,1,1)[12]           

...

 ARIMA(2,1,2)(0,1,0)[12]             : AIC=1580.543, Time=0.34 sec
 ARIMA(2,1,2)(0,1,1)[12]             : AIC=1545.251, Time=1.02 sec
 ARIMA(2,1,2)(1,1,0)[12]             : AIC=1493.543, Time=1.01 sec

Best model:  ARIMA(2,1,1)(1,1,0)[12]          
Total fit time: 56.337 seconds
91
 ARIMA(0,1,0)(0,1,0)[12]             : AIC=1720.319, Time=0.02 sec
 ARIMA(0,1,0)(0,1,1)[12]             : AIC=inf, Time=0.16 sec
 ARIMA(0,1,0)(0,1,2)[12]             : AIC=inf, Time=0.67 sec
 ARIMA(0,1,0)(1,1,0)[12]             : AIC=inf, Time=0.06 sec
 ARIMA(0,1,0)(1,1,1)[12]             : AIC=1451.281, Time=0.49 sec
 ARIMA(0,1,0)(1,1,2)[12]             : AIC=inf, Time=0.94 sec
 ARIMA(0,1,0)(2,1,0)[12]             : AIC=inf, Time=0.42 sec
 ARIMA(0,1,0)(2,1,1)[12]             : AIC=1450.144, Time=0.60 sec
 ARIMA(0,1,0)(2,1,2)[12]             : AIC=inf, Time=1.21 sec
 ARIMA(0,1,1)(0,1,0)[12]             : AIC=inf, Time=0.06 sec
 ARIMA(0,1,1)(0,1,1)[12]             : AIC=inf, Time=0.47 sec
 ARIMA(0,1,1)(0,1,2)

 ARIMA(2,1,1)(0,1,2)[12]             : AIC=inf, Time=1.54 sec
 ARIMA(2,1,1)(1,1,0)[12]             : AIC=inf, Time=nan sec
 ARIMA(2,1,1)(1,1,1)[12]             : AIC=1805.791, Time=1.23 sec
 ARIMA(2,1,1)(2,1,0)[12]             : AIC=inf, Time=1.25 sec
 ARIMA(2,1,2)(0,1,0)[12]             : AIC=1852.491, Time=0.11 sec
 ARIMA(2,1,2)(0,1,1)[12]             : AIC=1830.390, Time=0.32 sec
 ARIMA(2,1,2)(1,1,0)[12]             : AIC=inf, Time=nan sec

Best model:  ARIMA(1,1,0)(1,1,1)[12]          
Total fit time: 57.708 seconds
76
 ARIMA(0,1,0)(0,1,0)[12]             : AIC=1746.157, Time=0.02 sec
 ARIMA(0,1,0)(0,1,1)[12]             : AIC=inf, Time=0.18 sec
 ARIMA(0,1,0)(0,1,2)[12]             : AIC=inf, Time=0.59 sec
 ARIMA(0,1,0)(1,1,0)[12]             : AIC=inf, Time=0.08 sec
 ARIMA(0,1,0)(1,1,1)[12]             : AIC=inf, Time=0.25 sec
 ARIMA(0,1,0)(1,1,2)[12]             : AIC=inf, Time=0.65 sec
 ARIMA(0,1,0)(2,1,0)[12]             : AIC=inf, Time=0.42 sec
 ARIMA(0,1,0)(2,1,1)[12]        

 ARIMA(2,1,0)(2,1,0)[12]             : AIC=inf, Time=1.27 sec
 ARIMA(2,1,0)(2,1,1)[12]             : AIC=inf, Time=1.00 sec
 ARIMA(2,1,1)(0,1,0)[12]             : AIC=1778.257, Time=0.15 sec
 ARIMA(2,1,1)(0,1,1)[12]             : AIC=1713.395, Time=0.85 sec
 ARIMA(2,1,1)(0,1,2)[12]             : AIC=inf, Time=1.98 sec
 ARIMA(2,1,1)(1,1,0)[12]             : AIC=inf, Time=1.57 sec
 ARIMA(2,1,1)(1,1,1)[12]             : AIC=1709.119, Time=1.08 sec
 ARIMA(2,1,1)(2,1,0)[12]             : AIC=inf, Time=1.77 sec
 ARIMA(2,1,2)(0,1,0)[12]             : AIC=1753.170, Time=0.32 sec
 ARIMA(2,1,2)(0,1,1)[12]             : AIC=inf, Time=1.22 sec
 ARIMA(2,1,2)(1,1,0)[12]             : AIC=1695.582, Time=1.49 sec

Best model:  ARIMA(1,1,0)(1,1,1)[12]          
Total fit time: 65.648 seconds
90
 ARIMA(0,1,0)(0,1,0)[12]             : AIC=1559.954, Time=0.02 sec
 ARIMA(0,1,0)(0,1,1)[12]             : AIC=inf, Time=0.12 sec
 ARIMA(0,1,0)(0,1,2)[12]             : AIC=inf, Time=0.67 sec
 ARIMA(0,1,0)(1,1,0)

 ARIMA(2,1,0)(0,1,2)[12]             : AIC=inf, Time=1.02 sec
 ARIMA(2,1,0)(1,1,0)[12]             : AIC=inf, Time=0.97 sec
 ARIMA(2,1,0)(1,1,1)[12]             : AIC=1433.208, Time=1.02 sec
 ARIMA(2,1,0)(1,1,2)[12]             : AIC=inf, Time=4.82 sec
 ARIMA(2,1,0)(2,1,0)[12]             : AIC=inf, Time=0.68 sec
 ARIMA(2,1,0)(2,1,1)[12]             : AIC=inf, Time=0.62 sec
 ARIMA(2,1,1)(0,1,0)[12]             : AIC=inf, Time=0.14 sec
 ARIMA(2,1,1)(0,1,1)[12]             : AIC=inf, Time=0.71 sec
 ARIMA(2,1,1)(0,1,2)[12]             : AIC=inf, Time=1.41 sec
 ARIMA(2,1,1)(1,1,0)[12]             : AIC=inf, Time=nan sec
 ARIMA(2,1,1)(1,1,1)[12]             : AIC=inf, Time=1.20 sec
 ARIMA(2,1,1)(2,1,0)[12]             : AIC=inf, Time=1.60 sec
 ARIMA(2,1,2)(0,1,0)[12]             : AIC=inf, Time=0.31 sec
 ARIMA(2,1,2)(0,1,1)[12]             : AIC=1584.185, Time=0.98 sec
 ARIMA(2,1,2)(1,1,0)[12]             : AIC=inf, Time=0.46 sec

Best model:  ARIMA(2,1,0)(1,1,1)[12]          
Total fit tim

 ARIMA(1,1,2)(2,1,0)[12]             : AIC=inf, Time=0.55 sec
 ARIMA(2,1,0)(0,1,0)[12]             : AIC=1527.389, Time=0.03 sec
 ARIMA(2,1,0)(0,1,1)[12]             : AIC=inf, Time=0.57 sec
 ARIMA(2,1,0)(0,1,2)[12]             : AIC=inf, Time=1.16 sec
 ARIMA(2,1,0)(1,1,0)[12]             : AIC=inf, Time=0.39 sec
 ARIMA(2,1,0)(1,1,1)[12]             : AIC=1392.557, Time=0.45 sec
 ARIMA(2,1,0)(1,1,2)[12]             : AIC=inf, Time=1.83 sec
 ARIMA(2,1,0)(2,1,0)[12]             : AIC=inf, Time=1.46 sec
 ARIMA(2,1,0)(2,1,1)[12]             : AIC=inf, Time=0.96 sec
 ARIMA(2,1,1)(0,1,0)[12]             : AIC=inf, Time=0.17 sec
 ARIMA(2,1,1)(0,1,1)[12]             : AIC=inf, Time=0.72 sec
 ARIMA(2,1,1)(0,1,2)[12]             : AIC=inf, Time=1.26 sec
 ARIMA(2,1,1)(1,1,0)[12]             : AIC=1297.541, Time=0.78 sec
 ARIMA(2,1,1)(1,1,1)[12]             : AIC=inf, Time=2.31 sec
 ARIMA(2,1,1)(2,1,0)[12]             : AIC=inf, Time=2.66 sec
 ARIMA(2,1,2)(0,1,0)[12]             : AIC=inf, Time=0.